# 1. Preparation before running plans... 

## 1.1. Import Numpy and Matplotlib

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## 1.2. Create a RunEngine:

In [2]:
from bluesky import RunEngine
RE = RunEngine({})

## 1.3. Prepare Live Visualization¶

In [3]:
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

# Make plots update live while scans run.
from bluesky.utils import install_kicker
install_kicker()

## 1.4. Prepare Data Storage

In [4]:
from databroker import Broker
db = Broker.named('temp')

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)

1

##### This example makes a temporary database. Do not use it for important data. The data will become difficult to access once Python exits or the variable db is deleted. Running Broker.named('temp') a second time creates a fresh, separate temporary database.

## 1.5. Add a Progress Bar

In [5]:
from bluesky.utils import ProgressBarManager
RE.waiting_hook = ProgressBarManager()

# 2. Flow Experiment Plan

In [6]:
from ophyd import Component as Cpt, Device, EpicsMotor, EpicsSignal, EpicsSignalRO

Use simpleIOC in caproto as simulated EPICS

$ python -m caproto.ioc_examples.simple --prefix="simple:ultra:" --list-pvs

$ python -m caproto.ioc_examples.simple --prefix="simple:dds:" --list-pvs

In [7]:
### Two phdultra, two pump33dds  ==>  Each has tow syringe pumps. ==> Totla eight pumps.
# PVs = {'ultra1_A' : 'simple:A',
#        'ultra1_B' : 'simple:B',
#        'ultra2_A' : 'TBD',
#        'ultra2_B' : 'TBD',
#        'dds1_A' : 'TBD',
#        'dds1_B' : 'TBD',
#        'dds2_A' : 'TBD',
#        'dds2_B' : 'TBD',
#       }

In [8]:
# PV_list = list(PVs.keys())
# print(PV_list)

In [21]:
class ultra_dds(Device):
    rate = Cpt(EpicsSignal, 'A')
    pump = Cpt(EpicsSignal, 'B')

ultra1 = ultra_dds('simple:ultra:', name='ultra1')
dds1 = ultra_dds('simple:dds:', name='dds1')



# class ultra_dds(Device):
#     A = Cpt(EpicsSignal, 'A')
#     B = Cpt(EpicsSignal, 'B')

# class ultra2(Device):
#     A = Cpt(EpicsSignal, PV_list[2])
#     B = Cpt(EpicsSignal, PV_list[3])

# class dds1(Device):
#     A = Cpt(EpicsSignal, PV_list[4])
#     B = Cpt(EpicsSignal, PV_list[5])
    
# class dds2(Device):
#     A = Cpt(EpicsSignal, PV_list[6])
#     B = Cpt(EpicsSignal, PV_list[7])

In [22]:
ultra1.wait_for_connection()
dds1.wait_for_connection()

In [23]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable

RE = RunEngine()
token = RE.subscribe(LiveTable(["ultra1_A", "ultra1_B", "dds1_A", "dds1_B"]))

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)

1

In [16]:
from bluesky.plans import count

RE(count([dds1.B], num=3, delay=1))



+-----------+------------+------------+
|   seq_num |       time |     dds1_B |
+-----------+------------+------------+
|         1 | 17:29:09.4 |          2 |
|         2 | 17:29:10.4 |          2 |
|         3 | 17:29:11.4 |          2 |
+-----------+------------+------------+
generator count ['265731d3'] (scan num: 1)




('265731d3-2eda-4473-86a6-bd2c4d060ddd',)

In [20]:
ultra1.read()

OrderedDict([('ultra1_A', {'value': 5, 'timestamp': 1645049307.503526}),
             ('ultra1_B', {'value': 2.0, 'timestamp': 1645048040.024462})])

In [ ]:
dds1.read()

In [ ]:
ultra1.A.set(5).wait(timeout=1)

In [ ]:
RE(count([ultra1.A], num=3, delay=1))

In [17]:
header = db[-1]
header.table()

,time,dds1_B
seq_num,,
1,2022-02-16 22:29:09.456007957,2.0
2,2022-02-16 22:29:10.449524879,2.0
3,2022-02-16 22:29:11.450499058,2.0


In [18]:
from ophyd.device import DeviceStatus

In [19]:
DeviceStatus(ultra1)

DeviceStatus(device=ultra1, done=False, success=False)